In [ ]:
import pandas as pd
import os
import cv2

def clear_filepath(directory):
    for dirpath,_, filenames in os.walk(directory):
        for files in filenames:
            if not files.startswith("."):
                yield os.path.abspath(os.path.join(dirpath,files))

vid_list_path_all = list(clear_filepath("videos/to_analyse/OF_mir"))
if os.path.exists("OF_mirror/Timing_OF_mir.csv"):
    df_done = pd.read_csv("OF_mirror/Timing_OF_mir.csv")
    vids = list(pd.unique(df_done.Video))
    vid_list_path = []
    for vid_path in vid_list_path_all:
        vid_name = os.path.basename(vid_path)
        if vid_name not in vids:
            vid_list_path.append(vid_path)
else:
    vid_list_path = vid_list_path_all

for vid_path in vid_list_path:
    vid_name = os.path.basename(vid_path)
    print(vid_name)
    cap = cv2.VideoCapture(vid_path)
    df = pd.DataFrame({"Video":[vid_name]})
    count = 0
    door_open = False
    door_wrong = False
    box_open = False
    box_wrong = False
    while True:
        ret,frame = cap.read()
        if not ret:
            break
            
        count += 1
        cv2.imshow("Fish",frame)
        
        key= cv2.waitKey(1)
        if key == 100: # d key to save point when doors are fully open
            if door_wrong == False:
                door_open = True
                df["Door_open"] = count
        if key == 111: # o key to mark that the doors were open before the start of the video
            if door_open == False:
                door_wrong = True
                df["Door_open"] = "NA"
        
        if key == 32: # space key to save point when boxes fully lifted
            if box_wrong == False:
                box_open = True
                df["Box_opened"] = count
        if key == 98: # b key to mark that the box was fully lifted before the start of the video
            if box_open == False:
                box_wrong = True
                df["Box_opened"] = 0
        
        if key == 109: # m key to save point when mirror placed
            df["Mirror_drop"] = count
            break
        
        if key == 27:
            break
    
    if df.shape[1] == 4:
        if os.path.exists("OF_mirror/Timing_OF_mir.csv"):
            df.to_csv("OF_mirror/Timing_OF_mir.csv", index=False, header=False, mode="a")
        else:
            df.to_csv("OF_mirror/Timing_OF_mir.csv", index=False)
    else:
        print("Not all timing selected, data for this video not saved")

cv2.destroyAllWindows()
print("Done")